In [1]:
!pip install rawpy

menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 3.0.2 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.4.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as sl
import rawpy
import glob
import os
import gc

In [3]:
sess = tf.Session()

In [4]:
with tf.device('/device:GPU:0'):
    input_image = tf.placeholder(tf.float32, [None, None, None, 4])
    generated_image = tf.placeholder(tf.float32, [None, None, None, 3])

**custom layer**

In [5]:
def upsample_and_concat(c1, c2, output_channels, in_channels):
    pool_size = 2
    dcf = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    dc = tf.nn.conv2d_transpose(c1, dcf, tf.shape(c2), strides=[1, pool_size, pool_size, 1])

    output = tf.concat([dc, c2], 3)
    output.set_shape([None, None, None, output_channels * 2])

    return output

**custom activation**

In [6]:
def leaky_relu(x):
    return tf.maximum(x * 0.2, x) 

**Network**

How do we justify this complicated architecture ? Taken from github repo

In [7]:
# Unit 1
with tf.device('/device:GPU:0'):
    c1 = sl.conv2d(input_image, 32,[3,3], activation_fn=leaky_relu)
    c1 = sl.conv2d(c1, 32,[3,3], activation_fn=leaky_relu)
    p1 = sl.max_pool2d(c1, [2,2], padding='SAME')
# Unit 2
    c2 = sl.conv2d(p1, 64,[3,3], activation_fn=leaky_relu)
    c2 = sl.conv2d(c2, 64,[3,3], activation_fn=leaky_relu)
    p2 = sl.max_pool2d(c2, [2,2], padding='SAME')
# Unit 3
    c3 = sl.conv2d(p2, 128,[3,3], activation_fn=leaky_relu)
    c3 = sl.conv2d(c3, 128,[3,3], activation_fn=leaky_relu)
    p3 = sl.max_pool2d(c3, [2,2], padding='SAME')
# Unit 4
    c4 = sl.conv2d(p3, 256,[3,3], activation_fn=leaky_relu)
    c4 = sl.conv2d(c4, 256,[3,3], activation_fn=leaky_relu)
    p4 = sl.max_pool2d(c4, [2,2], padding='SAME')
# Unit 5
    c5 = sl.conv2d(p4, 512,[3,3], activation_fn=leaky_relu)
    c5 = sl.conv2d(c5, 512,[3,3], activation_fn=leaky_relu)
# Unit 6
    uc6 = upsample_and_concat(c5,c4,256,512)
    c6 = sl.conv2d(uc6, 256, [3,3], activation_fn=leaky_relu)
    c6 = sl.conv2d(c6, 256, [3,3], activation_fn=leaky_relu)
# Unit 7
    uc7 = upsample_and_concat(c6,c3,128,256)
    c7 = sl.conv2d(uc7, 128, [3,3], activation_fn=leaky_relu)
    c7 = sl.conv2d(c7, 128, [3,3], activation_fn=leaky_relu)
# Unit 8
    uc8 = upsample_and_concat(c7,c2,64,128)
    c8 = sl.conv2d(uc8, 64, [3,3], activation_fn=leaky_relu)
    c8 = sl.conv2d(c8, 64, [3,3], activation_fn=leaky_relu)
# Unit 9
    uc9 = upsample_and_concat(c8,c1,32,64)
    c9 = sl.conv2d(uc9, 32, [3,3], activation_fn=leaky_relu)
    c9 = sl.conv2d(c9, 32, [3,3], activation_fn=leaky_relu)
# Final Unit
    c10 = sl.conv2d(c9, 12, [1,1], activation_fn=None)
    output_image = tf.depth_to_space(c10,2)


**Loss Function**

In [8]:
with tf.device('/device:GPU:0'):
    loss = tf.reduce_mean(tf.abs(output_image - generated_image))

**Optimizer**

In [9]:
with tf.device('/device:GPU:0'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

In [10]:
sess.run(tf.global_variables_initializer())

In [11]:
input_image_dir = './Sony/short/'
generated_image_dir = './Sony/long/'
result_dir = './Result/'
train_ids = [int(os.path.basename(x)[0:5]) for x in glob.glob(generated_image_dir + '1*.ARW')]

In [12]:
len(train_ids)

50

**Converting Bayer image to 4 channel format**

In [13]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

**Initializing training lists**

In [14]:
generated_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

**Restoring training progress**

In [15]:
saver = tf.train.Saver()
saver.restore(sess, "my-test-model750l1.ckpt")

INFO:tensorflow:Restoring parameters from my-test-model750l1.ckpt


In [16]:
global_loss = 0

In [17]:
for uid in range(len(train_ids)):
    input_path = glob.glob(input_image_dir + '%05d_00*.ARW' % train_ids[uid])[0]
    generated_path = glob.glob(generated_image_dir + '%05d_00*.ARW' % train_ids[uid])[0]
        
    in_fn = os.path.basename(input_path)
    gt_fn = os.path.basename(generated_path)
    in_exposure = float(in_fn[9:-5])
    gt_exposure = float(gt_fn[9:-5])
    ratio = min(gt_exposure / in_exposure, 300)

    if input_images[str(ratio)[0:3]][uid] is None:
        raw = rawpy.imread(input_path)
        input_im = np.expand_dims(pack_raw(raw), axis=0) * ratio

        gt_raw = rawpy.imread(generated_path)
        im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        generated_im = np.expand_dims(np.float32(im / 65535.0), axis=0)

    input_im = np.minimum(input_im, 1.0)

    # Run image through tensorflow model
    _, current_loss, output = sess.run([optimizer, loss, output_image],
                                        feed_dict={input_image: input_im, generated_image: generated_im})
    output = np.minimum(np.maximum(output, 0), 1)
    global_loss += current_loss
    gc.collect()

In [18]:
global_loss/len(train_ids)

0.07999888822436332